http://python.jobbole.com/81683/

Python Decorators

## 1. 函数


函数通过def关键字、函数名和可选的参数列表定义。通过return关键字返回值。

In [1]:
def foo():
     return 1
foo()

1

## 2. 作用域

在python中，函数有自己的命名空间（作用域）。这意味着在函数内部碰到一个变量的时候函数会优先在自己的命名空间里面去寻找。

In [1]:
a_string = "This is a global variable"
def foo():
    a = 1
    print(locals())
# print(globals()) # doctest: +ELLIPSIS
foo()

{'a': 1}


## 3. 变量解析规则

在python的作用域规则里面，创建变量一定会一定会在当前作用域里创建一个变量，但是访问或者修改变量时会先在当前作用域查找变量，没有找到匹配变量的话会依次向上在闭合的作用域里面进行查看找。所以如果我们修改函数foo的实现让它打印全局的作用域里的变量也是可以的：

In [6]:
a_string = "This is a global variable"
b = 1
def foo():
    a = 1
    print(a_string) #1
    b = 3

# print(globals()) # doctest: +ELLIPSIS
foo()
b

This is a global variable


1

#### 在函数内部给全局变量赋值，结果却和我们想的不一样：

In [7]:
a_string = 'This is a global variable'
def foo():
    a_string = 'test' #2
    a = 1
    print(a_string)
# print(globals()) # doctest: +ELLIPSIS
foo()
print(a_string)

test
This is a global variable


我们能够看到，全局变量能够被访问到（如果是可变数据类型(像list,dict这些)甚至能够被更改）但是赋值不行。在函数内部的#1处，我们实际上新创建了一个局部变量，隐藏全局作用域中的同名变量。我们可以通过打印出局部命名空间中的内容得出这个结论。我们也能看到在#2处打印出来的变量a_string的值并没有改变。

#### 如果是可变数据类型(像list,dict这些)可以被更改

In [26]:
a_string = [1,2,3]
def foo():
    a_string[1] = 'test' #2
    a = 1
    print(a_string)
# print(globals()) # doctest: +ELLIPSIS
foo()
print(a_string)

[1, 'test', 3]
[1, 'test', 3]


## 4. 变量生存周期

In [8]:
def foo():
     x = 1
foo()
print(x) # 1


NameError: name 'x' is not defined

# 5. 函数参数



In [30]:
def foo(x):
     print(locals())
foo(1)


{'x': 1}


## 6. 嵌套函数



Python允许创建嵌套函数。这意味着我们可以在函数里面定义函数而且现有的作用域和变量生存周期依旧适用。



In [33]:
def outer():
    x = 1
    def inner():
        print(x) # 1
    inner() # 2
 
outer()

1


## 7. 函数是python世界里的一级类对象

In [35]:
def add(x, y):
     return x + y
def sub(x, y):
     return x - y
def apply(func, x, y): # 1
     return func(x, y) # 2
apply(add, 2, 1) # 3
# apply(sub, 2, 1)


3

#### 把函数当做返回值

In [17]:
def outer():
    def inner():
        print("Inside inner")
    return inner # 1
 
foo = outer() #2
foo()



Inside inner


In [5]:
def outer():
    x = "Inside inner"
    def inner():
        print(x)
    return inner # 1
 
foo = outer() #2
foo()

Inside inner


## 8. 闭包



闭包： 

　　在一个外函数中定义了一个内函数，内函数里运用了外函数的临时变量，并且外函数的返回值是内函数的引用。这样就构成了一个闭包。

In [60]:
def outer():
    x = 1
    def inner():

        print(x) # 1
    # The return value is a function    
    return inner
foo = outer()
foo()
foo.__closure__ # doctest: +ELLIPSIS


1


(<cell at 0x10687d2e8: int object at 0x1038b9550>,)

In [61]:

def outer(x):
    def inner():
        print(x) # 1
    return inner
print1 = outer(1)
print2 = outer(2)
print1()
print2()


1
2


一般情况下，在我们认知当中，如果一个函数结束，函数的内部所有东西都会释放掉，还给内存，局部变量都会消失。但是闭包是一种特殊情况，如果外函数在结束的时候发现有自己的临时变量将来会在内部函数中用到，就把这个临时变量绑定给了内部函数，然后自己再结束。

In [57]:
#闭包函数的实例
# outer是外部函数 a和b都是外函数的临时变量
def outer(a):
    b = 10
    # inner是内函数
    def inner():
        #在内函数中 用到了外函数的临时变量
        print(a+b)
    # 外函数的返回值是内函数的引用
    return inner

if __name__ == '__main__':
    # 在这里我们调用外函数传入参数5
    #此时外函数两个临时变量 a是5 b是10 ，并创建了内函数，然后把内函数的引用返回存给了demo
    # 外函数结束的时候发现内部函数将会用到自己的临时变量，这两个临时变量就不会释放，会绑定给这个内部函数
    demo = outer(5)
    # 我们调用内部函数，看一看内部函数是不是能使用外部函数的临时变量
    # demo存了外函数的返回值，也就是inner函数的引用，这里相当于执行inner函数
    demo() # 15

    # 一个函数，如果函数名后紧跟一对括号，
    # 相当于现在我就要调用这个函数，如果不跟括号，
    # 相当于只是一个函数的名字，里面存了函数所在位置的引用。
    demo2 = outer(7)
    demo2()#17

15
17


2 外函数把临时变量绑定给内函数：

　　按照我们正常的认知，一个函数结束的时候，会把自己的临时变量都释放还给内存，之后变量都不存在了。一般情况下，确实是这样的。但是闭包是一个特别的情况。外部函数发现，自己的临时变量会在将来的内部函数中用到，自己在结束的时候，返回内函数的同时，会把外函数的临时变量送给内函数绑定在一起。所以外函数已经结束了，调用内函数的时候仍然能够使用外函数的临时变量。

　　在我编写的实例中，我两次调用外部函数outer,分别传入的值是5和7。内部函数只定义了一次，我们发现调用的时候，内部函数是能识别外函数的临时变量是不一样的。python中一切都是对象，虽然函数我们只定义了一次，但是外函数在运行的时候，实际上是按照里面代码执行的，外函数里创建了一个函数，我们每次调用外函数，它都创建一个内函数，虽然代码一样，但是却创建了不同的对象，并且把每次传入的临时变量数值绑定给内函数，再把内函数引用返回。虽然内函数代码是一样的，但其实，我们每次调用外函数，都返回不同的实例对象的引用，他们的功能是一样的，但是它们实际上不是同一个函数对象。

闭包中内函数修改外函数局部变量：

在闭包内函数中，我们可以随意使用外函数绑定来的临时变量，但是如果我们想修改外函数临时变量数值的时候发现出问题了！咋回事捏？？！！（哇哇大哭）

在基本的python语法当中，一个函数可以随意读取全局数据，但是要修改全局数据的时候有两种方法:1 global 声明全局变量 2 全局变量是可变类型数据的时候可以修改

在闭包内函数也是类似的情况。在内函数中想修改闭包变量（外函数绑定给内函数的局部变量）的时候：

　　　　1 在python3中，可以用nonlocal 关键字声明 一个变量， 表示这个变量不是局部变量空间的变量，需要向上一层变量空间找这个变量。

　　　　2 在python2中，没有nonlocal这个关键字，我们可以把闭包变量改成可变类型数据进行修改，比如列表。

In [62]:
#修改闭包变量的实例
# outer是外部函数 a和b都是外函数的临时变量
def outer( a ):
    b = 10  # a和b都是闭包变量
    c = [a] #这里对应修改闭包变量的方法2
    # inner是内函数
    def inner():
        #内函数中想修改闭包变量
        # 方法1 nonlocal关键字声明
        nonlocal  b
        b+=1
        # 方法二，把闭包变量修改成可变数据类型 比如列表
        c[0] += 1
        print(c[0])
        print(b)
    # 外函数的返回值是内函数的引用
    return inner

if __name__ == '__main__':

    demo = outer(5)
    demo() # 6  11

6
11


使用闭包的过程中，一旦外函数被调用一次返回了内函数的引用，虽然每次调用内函数，是开启一个函数执行过后消亡，但是闭包变量实际上只有一份，每次开启内函数都在使用同一份闭包变量

In [24]:
#coding:utf8
def outer(x):
    def inner(y):
        nonlocal x
        x+=y
        return x
    return inner


a = outer(2)
print(a(1)) #11
print(a(3)) #14

3
6


闭包的作用：

1. 装饰器！！！装饰器是做什么的？？其中一个应用就是，我们工作中写了一个登录功能，我们想统计这个功能执行花了多长时间，我们可以用装饰器装饰这个登录模块，装饰器帮我们完成登录函数执行之前和之后取时间。

2. 面向对象！！！经历了上面的分析，我们发现外函数的临时变量送给了内函数。大家回想一下类对象的情况，对象有好多类似的属性和方法，所以我们创建类，用类创建出来的对象都具有相同的属性方法。闭包也是实现面向对象的方法之一。在python当中虽然我们不这样用，在其他编程语言入比如avaScript中，经常用闭包来实现面向对象编程

3. 实现单利模式！！ 其实这也是装饰器的应用。单利模式毕竟比较高大，，需要有一定项目经验才能理解单利模式到底是干啥用的，我们就不探讨了。

 


## 装饰器

装饰器是运用闭包的基本原理，对一个目标函数进行装饰。即是在执行一个目标函数之前、之后执行一些特定的事情。

In [74]:
#装饰器函数 传入目标函数做参数
def decorator( func ):
    #实际调用目标函数会发生调用inner，
    # 所以我们让inner接收不定参数，我们再把不定参数传给目标函数func
    #这样不论我们传入什么参数目标函数都能接收到！！
    def inner(*args, **kwargs):
        print("装饰器函数中。。目标函数执行之前的操作！！")
        # 我们设置一个变量接收目标函数的返回值，在inner结束的时候再把返回值返回去
        # python不同于其他语言，即使我们没有编写func的返回值，也会默认返回None，所以这里不会报错
        res = func(*args, **kwargs)
        print("装饰器函数中。。目标函数执行之后的操作！！")
        return res
    return inner
#这样编写的装饰器，在外部看来，我们就可以传入参数给目标函数
# 同时也可以正常接收目标函数返回的参数!!!

@decorator  #实际上会发生 destination = decorator( destination )
            # 把目标函数传入装饰器函数返回了inner给destination
            # 此后我们再调用destination 实际上调用了inner函数的一个对象
def destination(a):
    print("目标函数接受到参数:%s"%a)
    return "目标函数的返回值%s"%a

if __name__ == '__main__':
    # 这里实际上调用了inner函数的对象，我们传入参数和接收返回值都没有问题了！
    res = destination(10)
    # 打印一下返回值！
    print(res)


装饰器函数中。。目标函数执行之前的操作！！
目标函数接受到参数:10
装饰器函数中。。目标函数执行之后的操作！！
目标函数的返回值10


#### 多层装饰器嵌套

In [75]:
#多层装饰器的嵌套

#外层装饰器函数
def decorator_out(func ):
    def inner( *args , **kwargs ):
        print("外装饰器前置操作。。。。。。。。。")
        res = func( *args , **kwargs)
        print("外装饰器后置操作..............")
        return res
    return inner

#内层装饰器函数
def decorator_in( func ):
    def inner( *args , **kwargs ):
        print("内装饰器前置操作。。。。。。。。。")
        res = func( *args , **kwargs)
        print("内装饰器后置操作..............")
        return res
    return inner

#目标函数被两个装饰器装饰
'''
这里实际上会发生的情况是 login = decorator_out( decorator_in( destination )   )
先in装饰器装饰目标函数之后，把inner返回给destination,
然后 out装饰器 再对新的destination进行装饰，out里的func 存了 in装饰器装饰过的destination函数
    又把inner返回给destination

这时候再执行目标函数destination() 实际上
    1 执行out装饰器的inner() 执行到func的时候 执行in装饰器的inner
    2 在内装饰器inner中执行func才是原来的目标函数
    3 目标函数执行完跳出到in装饰器的inner
    4 in装饰器函数inner执行完 相当于out装饰器的func执行完 跳到out装饰器的inner中
    5 out装饰器执行结束，全部过程结束

'''
@decorator_out
@decorator_in
def destination():
    print("目标函数！")

if __name__ == '__main__':
    destination()


外装饰器前置操作。。。。。。。。。
内装饰器前置操作。。。。。。。。。
目标函数！
内装饰器后置操作..............
外装饰器后置操作..............


当多层装饰器嵌套的时候，实际上先内层装饰器装饰目标函数

外层装饰器会对内层装饰器装饰的结果进行再装饰

#### 可选择装饰器：

团队中一个人写了装饰器，大家一块用，之前的代码中，我写的装饰器，都是对目标函数执行之前和之后都有一些操作。如果某一个功能不需要之前或者之后的操作，只需要一个操作怎么办？？

这时候用到三层闭包嵌套。我们来一段代码理解一下

In [76]:
#我们传入一个flag决定是不是要执行目标函数之后的操作
def flagOperation( flag ):
    def decorator(func):
        def inner(*args , **kwargs):
            print("装饰器前置操作。。。。。。。。")
            res = func(*args , **kwargs)
            if flag :   #如果传入flag是真则执行后置操作 否则不执行
                print("我是后置操作")
            return res
        return inner
    return decorator



'''
@flagOperation(True) 这里实际上先执行了flagOperation(True) ，返回了decorator装饰器，
        并且把flag的值是True绑定给了他
    这时候用带着flag的decorator 对目标函数进行装饰
    相当于@decorator  带着一个flag
'''
@flagOperation(True)
def destination():
    print("目标函数！")

@flagOperation(False)
def desti2():
    print("目标函数2")

if __name__ == '__main__':
    destination()
    '''
    结果：
        装饰器前置操作。。。。。。。。
        目标函数！
        我是后置操作
    '''
    desti2()
    '''
    结果：
        装饰器前置操作。。。。。。。。
        目标函数2
    '''

装饰器前置操作。。。。。。。。
目标函数！
我是后置操作
装饰器前置操作。。。。。。。。
目标函数2


#### 类实现装饰器

In [21]:
#类装配期
class Decorator:
    def __init__(self,func):
        self.func = func
    def __call__(self):
        print("类装饰器前操作")
        self.func()
        print("类装饰器后操作")



@Decorator
def login():
    print("login now")
    
@Decorator    
def logoff():
    print("log off now")

if __name__ == '__main__':
    login()
    logoff()

类装饰器前操作
login now
类装饰器后操作
类装饰器前操作
log off now
类装饰器后操作
